## 目的

本コンペティションは、学生にエッセイを書かせ、その詳細な動作(入力や消去、移動など)からエッセイの採点結果を予想するテーブルコンペである。  
  
このnotebookでは、前処理から学習、提出までの流れをまとめる。  
なお、参考にしたnotebookは以下の通り。  
[https://www.kaggle.com/code/alexryzhkov/lgbm-and-nn-on-sentences/notebook](https://www.kaggle.com/code/alexryzhkov/lgbm-and-nn-on-sentences/notebook)

## 1. LightAutoMLのインストール
事前に[LightAutoML 038 dependecies](https://www.kaggle.com/code/alexryzhkov/lightautoml-038-dependecies)をAdd Dataしておく。

In [2]:
!pip install --no-index -U --find-links=/kaggle/input/lightautoml-038-dependecies lightautoml==0.3.8
!pip install --no-index -U --find-links=/kaggle/input/lightautoml-038-dependecies pandas==2.0.3

Looking in links: /kaggle/input/lightautoml-038-dependecies
Processing /kaggle/input/lightautoml-038-dependecies/lightautoml-0.3.8-py3-none-any.whl
Processing /kaggle/input/lightautoml-038-dependecies/AutoWoE-1.3.2-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/cmaes-0.10.0-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/joblib-1.2.0-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/json2html-1.3.0.tar.gz (from lightautoml==0.3.8)
  Preparing metadata (setup.py) ... done
Processing /kaggle/input/lightautoml-038-dependecies/lightgbm-3.2.1-py3-none-manylinux1_x86_64.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/poetry_core-1.8.1-py3-none-any.whl (from lightautoml

## 2. Import

In [3]:
%matplotlib inline
import gc
import os
import itertools
import pickle
import re
import time
from random import choice, choices
from functools import reduce
from tqdm import tqdm
from itertools import cycle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from functools import reduce
from itertools import cycle
from scipy import stats
from scipy.stats import skew, kurtosis
from sklearn import metrics, model_selection, preprocessing, linear_model, ensemble, decomposition, tree
import lightgbm as lgb
import copy

## 3. データの読み込み
初期に用意される入力データは、以下の4つが用意されている。
- train_logs.csv : キーロガーの記録(学習データ)
- train_scores.csv : エッセイの採点結果(学習データ)
- test_logs.csv : キーロガーの記録(テストデータ)
- sample_submission.csv : 提出用csvファイル

In [4]:
INPUT_DIR = '../input/linking-writing-processes-to-writing-quality'
train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')
ss_df = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

読み込んだデータの概要を確認する。  
各データの形状および先頭5行をみると、train_logsとtest_logsには「生徒のid」や「アクションの開始・終了の時間」、「アクションの種類(InputやRemoveなど)」などの情報が格納されている。  
一方で、train_scoresには「生徒のid」と「エッセイの採点結果」が格納されている。  
提出の形式は、「生徒のid」および「エッセイの採点結果」を提出するようだ。

In [5]:
print("train_logs shape : ", train_logs.shape)
print("train_scores shape : ", train_scores.shape)
print("test_logs shape : ", test_logs.shape)
print("sample_submission shape : ", ss_df.shape)

train_logs shape :  (8405898, 11)
train_scores shape :  (2471, 2)
test_logs shape :  (6, 11)
sample_submission shape :  (3, 2)


In [6]:
display(train_logs.head())
display(train_scores.head())

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0


In [7]:
display(test_logs.head())

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0


続いて、作成したエッセイの情報を読み込む。  
この情報はtrain_logsのキーロガー情報を結合し、各生徒が作成したエッセイを復元したデータである。実際に入力した文字は"q"に置き換えられているが、エッセイの長さなどの重要な情報が取得可能となる。  

<参考>  
[https://www.kaggle.com/code/hiarsl/feature-engineering-sentence-paragraph-features](https://www.kaggle.com/code/hiarsl/feature-engineering-sentence-paragraph-features)

In [8]:
train_essays = pd.read_csv('../input/writing-quality-challenge-constructed-essays/train_essays_02.csv')
train_essays.index = train_essays["Unnamed: 0"]
train_essays.index.name = None
train_essays.drop(columns=["Unnamed: 0"], inplace=True)
train_essays.head()

,essay
001519c8,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...
0022f953,"qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq..."
0042269b,qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...
0059420b,qq qqqqqqq qqqqqq qqqqqqqqqqqqq qqqq q qqqq qq...
0075873a,"qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq..."


また、テストデータについてもエッセイを復元する。  
テストデータのエッセイは、以下の関数を使用して新たにエッセイを復元する。
- processingInputs関数 : 各キーロガー記録からテキストの小部分を復元する関数(getEssays関数内で呼び出される。)
- getEssays関数 : エッセイを復元する関数

In [9]:
def getEssays(df):
    """
    エッセイの復元関数
    [input]
     df(pd.DataFrame) : キーロガー情報のデータフレーム
    [output]
     essayFrame(pd.DataFrame) : 復元したエッセイのデータフレーム
    """
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change']]
    textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']
    valCountsArr = textInputDf['id'].value_counts(sort=False).values
    lastIndex = 0
    essaySeries = pd.Series()
    for index, valCount in enumerate(valCountsArr):
        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change']].iloc[lastIndex : lastIndex + valCount]
        lastIndex += valCount
        essayText = ""
        for Input in currTextInput.values:
            if Input[0] == 'Replace':
                replaceTxt = Input[2].split(' => ')
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] +\
                essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue
            if Input[0] == 'Paste':
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue
            if Input[0] == 'Remove/Cut':
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue
            if "M" in Input[0]:
                croppedTxt = Input[0][10:]
                splitTxt = croppedTxt.split(' To ')
                valueArr = [item.split(', ') for item in splitTxt]
                moveData = (int(valueArr[0][0][1:]), 
                            int(valueArr[0][1][:-1]), 
                            int(valueArr[1][0][1:]), 
                            int(valueArr[1][1][:-1]))
                if moveData[0] != moveData[2]:
                    if moveData[0] < moveData[2]:
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] +\
                        essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] +\
                        essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
        essaySeries[index] = essayText
    essaySeries.index =  textInputDf['id'].unique()
    return pd.DataFrame(essaySeries, columns=['essay'])

In [10]:
# Features for test dataset
test_essays = getEssays(test_logs)
test_essays.head()

,essay
0000aaaa,
2222bbbb,qq
4444cccc,q


## 4.特徴量の作成

本節では、学習データを作成する準備として、各生徒のキーロガー情報から特徴量を作成していく。  
まず、最初の準備としては復元したエッセイ情報からエッセイの特徴に関する情報（文字数や平均文字数などの情報）を作成する。  
ここでは四分位数の第一四分位数と第三四分位数を求める関数と、復元したテキスト情報から、エッセイの情報を抜き出す関数を定義する。  
- q1関数 : 第一四分位数(25パーセンタイル)を返却する関数
- q3関数 : 第三四分位数(75パーセンタイル)を返却する関数
- split_essays_into_sentences関数 : 復元したエッセイを各文ごとに分割し、その文章の長さや単語数を返却する関数
- compute_sentence_aggregations関数 : 各文ごとに分割したエッセイ情報から文字数や平均文字数などの情報を返却する関数

In [11]:
# 第一四分位数と第三四分位数を返却する関数
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

In [12]:
AGGREGATIONS = ['count', 'mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', pd.DataFrame.kurt, 'sum']

def split_essays_into_sentences(df):
    """
    エッセイ情報の各文別データフレーム作成関数
    [input]
     df(pd.DataFrame) : 復元したエッセイのデータフレーム
    [output]
     essay_df(pd.DataFrame) : 復元したエッセイのデータフレーム
     
    復元したエッセイ情報をもとに文の最後にあるカンマ(.)をキーとしてエッセイを分割する。
    分割した各文の情報およびそれぞれの文章の長さや単語数の情報を追加して返却する。
    """
    essay_df = df
    essay_df['id'] = essay_df.index
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    essay_df = essay_df.explode('sent')
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
    return essay_df

def compute_sentence_aggregations(df):
    """
    エッセイ情報の統計情報取得関数
    [input]
     df(pd.DataFrame) : 復元したエッセイのデータフレーム
    [output]
     sent_agg_df(pd.DataFrame) : 復元したエッセイのデータフレーム
     
    カンマ(.)をキーとして分割したエッセイ情報のデータフレームから、生徒ごとの統計情報(平均や分散など)を
    取得し返却する。
    """
    sent_agg_df = pd.concat(
        [df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    )
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    return sent_agg_df

In [13]:
# Word features for train dataset
train_sent_df = split_essays_into_sentences(train_essays)
train_sent_agg_df = compute_sentence_aggregations(train_sent_df)

この関数を実行することで、各エッセイにおける各文の統計量が特徴量として取得できる。

In [14]:
display(train_sent_agg_df.head())

,sent_count,sent_len_mean,sent_len_std,sent_len_min,sent_len_max,sent_len_first,sent_len_last,sent_len_sem,sent_len_q1,sent_len_median,...,sent_word_count_first,sent_word_count_last,sent_word_count_sem,sent_word_count_q1,sent_word_count_median,sent_word_count_q3,sent_word_count_skew,sent_word_count_kurt,sent_word_count_sum,id
0,14,106.142857,41.128050,31,196,31,89,10.991934,75.5,119.5,...,6,16,1.736577,12.25,21.0,22.00,-0.506007,-0.526754,256,001519c8
1,15,107.666667,64.713287,19,226,19,143,16.708899,56.5,92.0,...,3,30,3.269872,12.00,20.0,31.00,0.391857,-0.935036,325,0022f953
2,19,133.842105,33.480115,73,189,139,161,7.680865,108.0,139.0,...,21,26,1.207599,17.50,21.0,26.50,-0.242560,-1.171619,408,0042269b
3,13,86.846154,33.195999,39,144,99,80,9.206914,62.0,80.0,...,17,14,1.800997,11.00,15.0,18.00,0.656055,-0.538051,208,0059420b
4,16,86.812500,44.094170,22,182,75,22,11.023543,60.0,74.0,...,11,3,2.166927,11.00,12.5,18.25,1.148513,0.888421,255,0075873a


次に、

In [15]:
def split_essays_into_paragraphs(df):
    essay_df = df
    essay_df['id'] = essay_df.index
    essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
    essay_df = essay_df.explode('paragraph')
    # Number of characters in paragraphs
    essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
    # Number of words in paragraphs
    essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
    return essay_df

def compute_paragraph_aggregations(df):
    paragraph_agg_df = pd.concat(
        [df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    ) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df

In [17]:
# Paragraph features for train dataset
train_paragraph_df = split_essays_into_paragraphs(train_essays)
train_paragraph_agg_df = compute_paragraph_aggregations(train_paragraph_df)

In [19]:
display(train_paragraph_df.head())
display(train_paragraph_agg_df.head())

,essay,id,sent,paragraph,paragraph_len,paragraph_word_count
0,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...,001519c8,"[qqqqqqqqq qq qqqqq qq qqqq qqqq, qqqqqq qqq...",qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...,390,71
1,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...,001519c8,"[qqqqqqqqq qq qqqqq qq qqqq qqqq, qqqqqq qqq...",qq qq qqqq qqqq qqq qqqqqqqqq qqq qqqqqqq qq q...,654,112
2,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...,001519c8,"[qqqqqqqqq qq qqqqq qq qqqq qqqq, qqqqqq qqq...","qqqq qqq qqqqqq qqqqqqqqqq qqqqqqqqq qqqqq, qq...",480,86
3,"qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq...",0022f953,"[qqqq qq qqqqqqqqqqq , qq qq qqq qqq qqq, qqq...","qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq...",240,53
4,"qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq...",0022f953,"[qqqq qq qqqqqqqqqqq , qq qq qqq qqq qqq, qqq...",qqqqqq qq qqqq qqq qqq qqqq qqq qqqqqq qq...,462,96


,paragraph_count,paragraph_len_mean,paragraph_len_std,paragraph_len_min,paragraph_len_max,paragraph_len_first,paragraph_len_last,paragraph_len_sem,paragraph_len_q1,paragraph_len_median,...,paragraph_word_count_first,paragraph_word_count_last,paragraph_word_count_sem,paragraph_word_count_q1,paragraph_word_count_median,paragraph_word_count_q3,paragraph_word_count_skew,paragraph_word_count_kurt,paragraph_word_count_sum,id
0,3,508.000000,134.208793,390,654,390,480,77.485483,435.00,480.0,...,71,86,11.976829,78.50,86.0,99.00,0.770543,NaN,269,001519c8
1,6,278.166667,98.554384,176,462,240,284,40.234659,228.75,261.0,...,53,60,8.316316,47.75,56.5,62.25,1.299614,2.342703,355,0022f953
2,6,429.500000,101.087586,296,568,491,296,41.268834,356.75,444.5,...,79,45,6.926599,55.50,73.5,78.75,-0.502908,-1.536764,410,0042269b
3,3,384.000000,56.471232,347,449,347,356,32.603681,351.50,356.0,...,62,65,5.897269,63.50,65.0,73.00,1.565482,NaN,208,0059420b
4,5,283.400000,232.336609,23,627,351,23,103.904090,124.00,292.0,...,61,3,18.706683,26.00,52.0,61.00,0.686760,0.722916,256,0075873a
